In [4]:
import os
import re
import numpy as np
import matplotlib.pyplot as plt

path='DataSG056'
file_path=[]
for filename in os.listdir(path):
    file_path.append((os.path.join(path,filename)))


### read data
* first 6 columns are dates `2021 04 01 00 00 00`
* 7th column -> G1-F60 in volt 
* 8th column -> G2-F60 in volt
* 9th column -> Br1-F60
* 10th column -> Br2-F60

In [37]:
data=np.array([])
for address in file_path:
    if data.size==0:
        data=np.loadtxt(address,skiprows=43,unpack=True)
    else:
        nd=np.loadtxt(address,skiprows=43,unpack=True)
        data=np.column_stack((data,nd))
print(data[6,:])

[-4.0750851 -4.0757518 -4.0764057 ... -3.2395485 -3.2484566 -3.258126 ]


## convert measurement
volt -> $\mu$gal

In [39]:
cal1=-40.11
cal2=-80.0699
data[6,:]=cal1*data[6,:]
data[7,:]=cal2*data[7,:]